**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [2]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-om11a2ne
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-om11a2ne
  Resolved https://github.com/Kaggle/learntools.git to commit 55574bf6512577688c9d0bcde665683903594910
  Preparing metadata (setup.py) ... done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268951 sha256=04e385803699267f28a89b41e7b25910b53150f36e82e75591ce805e20cf5ad5
  Stored in directory: /tmp/pip-ephem-wheel-cache-onops_ne/wheels/2f/6c/3c/aa9f50cfb5a862157cb4c7a5b34881828cf45404698255dced
Successfully built learntools
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [3]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,b3ec3cef49922433e4a5d8c74998af060224e308,e3dfdc6273b53e7bea4294c159fb810c907ffcd2aba9ba...,2015-01-24 21:00:00+00:00,2015-01-24 21:00:00+00:00,240,0.6,1.703108e+10,1.703108e+10,8,8,...,0.0,4.65,Cash,Dispatch Taxi Affiliation,41.907520,-87.626659,POINT (-87.6266589003 41.90752007470001),41.907492,-87.635760,POINT (-87.6357600901 41.9074919303)
1,7ded635e0822f2b6d854744d69caf601d8578fe3,19e804a8eab9224b352e6a384007418b519864a3e7c2d9...,2015-02-04 04:45:00+00:00,2015-02-04 05:00:00+00:00,540,2.9,NaN,NaN,22,24,...,3.0,12.25,Cash,Taxi Affiliation Services,41.922761,-87.699155,POINT (-87.69915534320002 41.9227606205),41.901207,-87.676356,POINT (-87.6763559892 41.90120699410001)
2,910e87798340a2b3e1739524f3b2ce9b3e6c62d5,1be31ca6ff375ee0a1a646ffeb90536c2a12453142c482...,2015-03-27 20:30:00+00:00,2015-03-27 20:45:00+00:00,600,1.7,1.703108e+10,1.703108e+10,8,8,...,0.0,7.65,Cash,Taxi Affiliation Services,41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),41.909496,-87.630964,POINT (-87.630963601 41.9094956686)
3,89717b37316bb8a61c27335d56e4b22110b11e16,ea9c7f865233f880e5f00abb728092901eeaf52c85a8c1...,2015-01-29 16:00:00+00:00,2015-01-29 16:15:00+00:00,240,0.0,1.703132e+10,1.703108e+10,32,8,...,0.0,5.85,Cash,Choice Taxi Association,41.877406,-87.621972,POINT (-87.6219716519 41.8774061234),41.890922,-87.618868,POINT (-87.6188683546 41.8909220259)
4,9c7429d65b397e01c79d8cc739cc3abaeac6ab84,0d61c4f9c8cb8d280fce388789bd7e26d2a692c0e510ca...,2014-12-13 15:30:00+00:00,2014-12-13 15:45:00+00:00,1200,2.7,1.703132e+10,1.703108e+10,32,8,...,1.0,11.45,Cash,Northwest Management LLC,41.877406,-87.621972,POINT (-87.6219716519 41.8774061234),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365)


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [10]:
# Fill in the blank below
avg_num_trips_query = """
WITH trips_by_day AS
(
    SELECT
        DATE(trip_start_timestamp) AS trip_date,
        COUNT(*) as num_trips
    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
    GROUP BY trip_date
    ORDER BY trip_date
)
SELECT
    trip_date,
    AVG(num_trips) OVER (
        ORDER BY trip_date
        ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
    ) AS avg_num_trips
FROM trips_by_day
"""

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-02-27,92965.142857
1,2016-01-13,85459.285714
2,2016-01-27,81378.428571
3,2016-03-24,86435.714286
4,2016-02-20,89393.428571


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [18]:
# Amend the query below
trip_number_query = """
SELECT
    pickup_community_area,
    trip_start_timestamp,
    trip_end_timestamp,
    RANK() OVER(
        PARTITION BY pickup_community_area
        ORDER BY trip_start_timestamp
    ) AS trip_number
FROM
    `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
    DATE(trip_start_timestamp) = '2013-10-03'
"""

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,15.0,2013-10-03 00:00:00+00:00,2013-10-03 01:30:00+00:00,1
1,15.0,2013-10-03 00:00:00+00:00,2013-10-03 00:00:00+00:00,1
2,15.0,2013-10-03 00:30:00+00:00,2013-10-03 00:45:00+00:00,3
3,15.0,2013-10-03 01:30:00+00:00,2013-10-03 01:30:00+00:00,4
4,15.0,2013-10-03 03:00:00+00:00,2013-10-03 03:00:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [22]:
# Fill in the blanks below
break_time_query = """
SELECT
    taxi_id,
    trip_start_timestamp,
    trip_end_timestamp,
    TIMESTAMP_DIFF(
        trip_start_timestamp,
        LAG(trip_end_timestamp, 1) OVER (
            PARTITION BY taxi_id
            ORDER BY trip_start_timestamp
        ),
        MINUTE) as prev_break
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE DATE(trip_start_timestamp) = '2013-10-03' 
"""

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,01271057890af916c8ee4890c076e44fb6ac327a1868de...,2013-10-03 16:30:00+00:00,2013-10-03 16:45:00+00:00,270.0
1,01271057890af916c8ee4890c076e44fb6ac327a1868de...,2013-10-03 23:30:00+00:00,2013-10-03 23:15:00+00:00,195.0
2,0e459cbb5049e55828a11cd1deb08a2d25359dfd6f1a77...,2013-10-03 05:45:00+00:00,2013-10-03 06:00:00+00:00,270.0
3,0e459cbb5049e55828a11cd1deb08a2d25359dfd6f1a77...,2013-10-03 13:30:00+00:00,2013-10-03 13:30:00+00:00,285.0
4,10e8a565ac8829bd9caa44f57e9831fd8b4ff374e4d666...,2013-10-03 14:00:00+00:00,2013-10-03 13:45:00+00:00,825.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*